# 2 — ROS2 Architecture & Core Concepts

> Deep dive into Nodes, Topics, Services, Actions, and Parameters — the building blocks of every ROS2 system.

**Goal:** Understand *how* ROS2 components communicate so that when you write code in later notebooks, you know exactly what's happening under the hood.

In [ ]:
#| default_exp architecture

## 2.1 The Computation Graph

A ROS2 system is a **graph** of processes that communicate over well-defined channels:

```
  ┌──────────┐       /scan        ┌───────────┐      /map       ┌──────────┐
  │  LIDAR   │──────────────────►  │   SLAM    │───────────────► │  Nav2    │
  │  Driver  │                     │   Node    │                 │ Planner  │
  └──────────┘                     └───────────┘                 └────┬─────┘
                                                                      │
  ┌──────────┐    /camera/image    ┌───────────┐                 /cmd_vel
  │  Camera  │──────────────────►  │ Detector  │                      │
  │  Driver  │                     │   Node    │                 ┌────▼─────┐
  └──────────┘                     └───────────┘                 │  Motor   │
                                                                 │ Controller│
                                                                 └──────────┘
```

Every box is a **Node**. Every arrow is a **Topic**. This is the **computation graph**.

### Useful CLI Commands to Inspect the Graph

```bash
# List all running nodes
ros2 node list

# List all active topics
ros2 topic list

# See the graph visually
rqt_graph
```

## 2.2 Nodes — The Unit of Computation

A **node** is a single process that does one thing well:

- `camera_driver` — reads frames from hardware, publishes images
- `object_detector` — subscribes to images, publishes detections
- `motor_controller` — subscribes to velocity commands, drives motors

### Key Properties

| Property | Description |
|----------|------------|
| **Name** | Unique string, e.g., `camera_driver`. Set when you create the node. |
| **Namespace** | Optional prefix for grouping, e.g., `/robot1/camera_driver` |
| **Lifecycle** | Nodes can be Unconfigured → Inactive → Active → Finalized |
| **Executor** | Manages callbacks (single-threaded or multi-threaded) |

### Why One Node Per Task?

1. **Fault isolation** — if the detector crashes, the camera driver keeps working
2. **Reusability** — the same camera driver works with any detector
3. **Distributed** — run CPU-heavy nodes on a powerful machine, lightweight nodes on the robot
4. **Testability** — test each node independently with mock data

### CLI Inspection

```bash
# Show info about a specific node
ros2 node info /camera_driver

# Output shows:
#   Subscribers: (topics this node listens to)
#   Publishers:  (topics this node writes to)
#   Services:    (services this node offers/calls)
```

## 2.3 Topics — Asynchronous Data Streams

A **topic** is a named channel where nodes send and receive **messages**.

```
Publisher Node  ───── /topic_name ─────►  Subscriber Node(s)
                   (message type)
```

### Characteristics

| Feature | Detail |
|---------|--------|
| **Direction** | Unidirectional: publisher → subscriber |
| **Cardinality** | Many-to-many: N publishers, M subscribers on the same topic |
| **Coupling** | Loose: publishers don't know who subscribes |
| **Delivery** | Continuous stream (not request/response) |

### Messages & Types

Every topic has a **message type** — a structured data format defined in `.msg` files:

```
# Example: sensor_msgs/msg/Image
std_msgs/Header header
uint32 height
uint32 width
string encoding          # e.g., "bgr8", "rgb8", "mono8"
uint8 is_bigendian
uint32 step              # row length in bytes
uint8[] data             # the actual pixel data
```

Common message packages:

| Package | Common Messages | Used For |
|---------|----------------|----------|
| `std_msgs` | `String`, `Int32`, `Float64`, `Bool` | Simple data |
| `sensor_msgs` | `Image`, `LaserScan`, `Imu`, `PointCloud2` | Sensor data |
| `geometry_msgs` | `Twist`, `Pose`, `Transform` | Movement & position |
| `nav_msgs` | `Odometry`, `OccupancyGrid`, `Path` | Navigation |

### CLI Inspection

```bash
# List topics
ros2 topic list
ros2 topic list -t    # with types

# See data flowing on a topic
ros2 topic echo /camera/image

# Check publish rate
ros2 topic hz /camera/image

# Show message type definition
ros2 interface show sensor_msgs/msg/Image
```

## 2.4 Quality of Service (QoS) — Controlling Reliability

QoS policies let you tune how messages are delivered. This is a **critical** concept that many beginners miss.

| Policy | Options | When to use |
|--------|---------|-------------|
| **Reliability** | `RELIABLE` / `BEST_EFFORT` | Camera images → BEST_EFFORT (drop frames if slow). Commands → RELIABLE (never lose a stop command) |
| **Durability** | `VOLATILE` / `TRANSIENT_LOCAL` | TRANSIENT_LOCAL means late subscribers get the last value (good for `/map`) |
| **History** | `KEEP_LAST(N)` / `KEEP_ALL` | KEEP_LAST(1) for images (only want newest). KEEP_ALL for logging. |
| **Depth** | Integer | Queue size for KEEP_LAST |

### Compatibility Rules

**Publisher and subscriber QoS must be compatible** or the connection silently fails!

| Publisher | Subscriber | Works? |
|-----------|-----------|--------|
| RELIABLE | RELIABLE | ✅ |
| BEST_EFFORT | BEST_EFFORT | ✅ |
| RELIABLE | BEST_EFFORT | ✅ |
| BEST_EFFORT | RELIABLE | ❌ **Fails silently!** |

> **Tip:** If your subscriber receives no data, QoS mismatch is the #1 cause. Use `ros2 topic info /topic_name --verbose` to check.

## 2.5 Services — Synchronous Request/Response

When you need a **one-shot response** (not a stream), use a Service:

```
Client  ──── request ────►  Service Server
        ◄─── response ───
```

### Example: Set a parameter on a camera

```
# Service type: rcl_interfaces/srv/SetParameters
# Request:  parameters: [{name: 'exposure', value: {type: 3, double_value: 0.01}}]
# Response: results: [{successful: true}]
```

### When Topics vs Services?

| Use Case | Pattern |
|----------|--------|
| Continuous sensor data | **Topic** |
| One-time query ("what's the battery level?") | **Service** |
| Trigger an action ("take a snapshot") | **Service** |
| Change configuration | **Service** or **Parameter** |

### CLI

```bash
ros2 service list
ros2 service type /set_parameters
ros2 service call /set_parameters rcl_interfaces/srv/SetParameters \
  "{parameters: [{name: 'exposure', value: {type: 3, double_value: 0.01}}]}"
```

## 2.6 Actions — Long-Running Tasks with Feedback

Actions combine topics and services for **long-running, cancellable tasks**:

```
Client  ──── goal ────────►  Action Server
        ◄─── feedback ────   (periodic updates)
        ◄─── result ──────   (when complete)
```

### Real-World Example: Navigate to a Point

```
Goal:     "Go to position (3.0, 5.0)"
Feedback: "Distance remaining: 2.1m" → "1.3m" → "0.5m"
Result:   "Arrived at position (3.0, 5.0)"
```

You can also **cancel** an action mid-way (e.g., obstacle detected, abort!).

### When to Use Actions

| Scenario | Pattern |
|----------|--------|
| Navigate to waypoint | **Action** |
| Pick up an object | **Action** |
| Camera exposure (quick set) | **Service** |
| Stream images | **Topic** |

### CLI

```bash
ros2 action list
ros2 action info /navigate_to_pose
ros2 action send_goal /navigate_to_pose nav2_msgs/action/NavigateToPose \
  "{pose: {header: {frame_id: 'map'}, pose: {position: {x: 3.0, y: 5.0}}}}"
```

## 2.7 Parameters — Runtime Configuration

Parameters are **key-value pairs** that nodes expose for runtime tuning:

```bash
# List parameters of a node
ros2 param list /camera_driver

# Get a value
ros2 param get /camera_driver frame_rate

# Set a value
ros2 param set /camera_driver frame_rate 30

# Dump all parameters to a YAML file
ros2 param dump /camera_driver
```

Parameters let you change behaviour **without recompiling or restarting** the node (if the node supports dynamic reconfiguration).

## 2.8 DDS — The Middleware Under the Hood

**DDS (Data Distribution Service)** is the industry standard protocol that ROS2 uses for all communication. You rarely interact with DDS directly, but understanding it helps:

```
┌─────────────────────────────────────────────┐
│              Your Python Code                │
│   rclpy.create_node('my_node')              │
├─────────────────────────────────────────────┤
│           ROS2 Client Library                │
│   rcl → rmw (ROS Middleware Interface)       │
├─────────────────────────────────────────────┤
│           DDS Implementation                 │
│   CycloneDDS / FastDDS / ConnextDDS          │
├─────────────────────────────────────────────┤
│           UDP Multicast / Unicast            │
│   (automatic peer discovery on LAN)          │
└─────────────────────────────────────────────┘
```

### Key DDS Features

- **Auto-discovery:** Nodes on the same network automatically find each other — no master/broker needed
- **QoS:** Reliability, durability, and deadline policies (covered in 2.4)
- **Domain ID:** An integer that isolates groups of nodes (like separate channels)

### Common DDS Implementations

| DDS | Default in | Strengths |
|-----|-----------|----------|
| **CycloneDDS** | Jazzy | Lightweight, open-source, good for most use cases |
| **FastDDS** | Humble | Feature-rich, well-tested |
| **ConnextDDS** | — | Commercial, real-time certified |

You can switch DDS implementations with an environment variable:

```bash
export RMW_IMPLEMENTATION=rmw_cyclonedds_cpp
```

## 2.9 TF2 — The Transform Library

Robots have many coordinate frames (base, camera, LIDAR, each wheel…). **TF2** keeps track of the relationships:

```
         map
          │
        odom
          │
      base_link
       /     \
  camera    lidar
  _link     _link
```

TF2 answers: "Where is the camera relative to the base?" or "Where is the detected object in the map frame?"

```bash
# View the TF tree
ros2 run tf2_tools view_frames
# Opens a PDF showing all transforms

# Look up a specific transform
ros2 run tf2_ros tf2_echo base_link camera_link
```

> **We won't code TF2 in this course**, but it's essential to understand that it exists. Any camera or sensor node publishes a TF transform from the sensor to the robot base.

## 2.10 Launch Files — Starting Everything at Once

In production, you don't start nodes one by one. **Launch files** start multiple nodes with configuration:

```python
# example_launch.py
from launch import LaunchDescription
from launch_ros.actions import Node

def generate_launch_description():
    return LaunchDescription([
        Node(
            package='usb_cam',
            executable='usb_cam_node_exe',
            name='camera',
            parameters=[{'video_device': '/dev/video0',
                         'framerate': 30.0}]
        ),
        Node(
            package='my_package',
            executable='detector_node',
            name='detector',
            remappings=[('/image_raw', '/camera/image_raw')]
        ),
    ])
```

```bash
# Run a launch file:
ros2 launch my_package example_launch.py
```

Launch files can also:
- Include other launch files
- Conditionally start nodes
- Pass arguments from the command line
- Load YAML parameter files

## 2.11 Summary — The Mental Model

```
ROS2 = Nodes + Communication + Tools

Communication:
  Topics    → continuous streams (camera images, sensor data)
  Services  → quick request/response (set parameter, query state)
  Actions   → long tasks with feedback (navigate, pick object)
  Parameters → runtime configuration (frame_rate, threshold)

Tools:
  colcon    → build
  ros2 CLI  → inspect, debug
  RViz2     → visualize
  rqt       → GUI introspection
  rosbag2   → record & replay
  launch    → orchestrate
```

### Quick Reference: CLI Cheat Sheet

| Command | What it does |
|---------|-------------|
| `ros2 node list` | List running nodes |
| `ros2 topic list -t` | List topics with types |
| `ros2 topic echo /topic` | Print messages on a topic |
| `ros2 topic hz /topic` | Show publish rate |
| `ros2 topic info /topic --verbose` | Show QoS policies |
| `ros2 service list` | List services |
| `ros2 service call ...` | Call a service |
| `ros2 action list` | List actions |
| `ros2 param list /node` | List parameters |
| `ros2 interface show <type>` | Show message/service/action definition |
| `rqt_graph` | Visualize the computation graph |
| `ros2 bag record -a` | Record all topics |

---

**Next →** [Notebook 03: Workspace & Package Setup](03_workspace_setup.ipynb)